In [1]:
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6 libfontconfig1 libxrender1

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Hit:1 http://archive.ubuntu.com/ubuntu xenial InRelease
Hit:2 http://security.ubuntu.com/ubuntu xenial-security InRelease       
Hit:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease         
Hit:4 http://archive.ubuntu.com/ubuntu xenial-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
16 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
libxrender1 is already the newest version (1:0.9.9-0ubuntu1).
libfontconfig1 is already the newest version (2.11.94-0ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [2]:
import numpy as np
import os
import sys
import tensorflow as tf
from collections import OrderedDict
import re
from PIL import Image
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import pickle
import cv2

print("Tensorflow version: {}".format(tf.__version__))

Tensorflow version: 1.11.0


# Load Model and Graph

In [3]:
"""
PB: the frozen model
pb.txt: the label map file
"""
PB_NAME = "ssd_inception_v2_m_0.2.1"
#PATH_TO_PB = "/Users/jiankaiwang/devops/Fruit_Recognition/models/ssd_inception_v2/res/m200000/{}.pb".format(PB_NAME)
#PATH_TO_LABELS = "/Users/jiankaiwang/devops/Fruit_Recognition/models/ssd_inception_v2/image_label.pbtxt"
PATH_TO_PB = "/notebooks/devops/Auxiliary_Operations/model/inception_single_0.1/inception_single_0.1.pb"
PATH_TO_LABELS = "/notebooks/devops/Auxiliary_Operations/model/inception_single_0.1/inception_single_0.1.txt"

PATH_TO_PICKLE_DIR = "/notebooks/devops/Auxiliary_Operations/model/inception_single_0.1"

if not os.path.exists(PATH_TO_PB): raise FileNotFoundError("PB is not found.")
if not os.path.exists(PATH_TO_LABELS): raise FileNotFoundError("Label is not found")
if not os.path.exists(PATH_TO_PICKLE_DIR): raise FileNotFoundError("PICKLE PATH is not found")

## load frozen graph 

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_PB, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Show all operation names

In [5]:
def show_operation_names(count=10):
    with detection_graph.as_default():
        with tf.Session() as sess:
            opts = tf.get_default_graph().get_operations()
            for opt in opts[:count]: 
                for output in opt.outputs: print(output.name)
            print("...")
            for opt in opts[-count:]: 
                for output in opt.outputs: print(output.name)
                    
show_operation_names(10)

Placeholder:0
module/InceptionV3/Conv2d_1a_3x3/weights:0
module/InceptionV3/Conv2d_1a_3x3/BatchNorm/beta:0
module/InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean:0
module/InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_variance:0
module/InceptionV3/Conv2d_2a_3x3/weights:0
module/InceptionV3/Conv2d_2a_3x3/BatchNorm/beta:0
module/InceptionV3/Conv2d_2a_3x3/BatchNorm/moving_mean:0
module/InceptionV3/Conv2d_2a_3x3/BatchNorm/moving_variance:0
module/InceptionV3/Conv2d_2b_3x3/weights:0
...
module_apply_default/InceptionV3/Logits/GlobalPool:0
module_apply_default/hub_output/feature_vector/SpatialSqueeze:0
input/BottleneckInputPlaceholder:0
final_retrain_ops/weights/final_weights:0
final_retrain_ops/weights/final_weights/read:0
final_retrain_ops/biases/final_biases:0
final_retrain_ops/biases/final_biases/read:0
final_retrain_ops/Wx_plus_b/MatMul:0
final_retrain_ops/Wx_plus_b/add:0
final_result:0


## Load Label file

In [6]:
category_index = OrderedDict()
name_indx = OrderedDict()
count = 1
with open(PATH_TO_LABELS, "r") as fin:
    tmpData = ""
    for line in fin:
        tmpData = line.strip()
        category_index[count] = tmpData
        name_indx[tmpData] = count
        count += 1
    print(category_index)
    print(name_indx)

OrderedDict([(1, 'hole'), (2, 'screw')])
OrderedDict([('hole', 1), ('screw', 2)])


# Inference 

In [7]:
def inference_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # handle input and output tensor
            opts = tf.get_default_graph().get_operations()
            all_tensorflow_names = { output.name for opt in opts for output in opt.outputs }
            tensor_dict = {}
            for key in ['final_result']:
                tensor_name = key + ':0'
                if tensor_name in all_tensorflow_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
                    
            # run for single image            
            # input
            image_tensor = tf.get_default_graph().get_tensor_by_name('Placeholder:0')
            
            # inference
            output_dict = sess.run(tensor_dict, feed_dict={image_tensor: np.expand_dims(image, 0)})
            
            # convert data type float32 to appropriate
            output_dict['final_result'] = output_dict['final_result']
            
        return output_dict

In [8]:
def single_image(imagePath):
    image_path = imagePath
    if not os.path.exists(image_path): raise FileNotFoundError("{} not found.".format(image_path))
        
    image = cv2.imread(image_path)
    image = cv2.resize(image, (299, 299), interpolation=cv2.INTER_CUBIC)
    image_np = image[:,:,::-1]
    output_dict = inference_single_image(image_np, detection_graph)
    
    return output_dict

In [9]:
#image_path = '/Users/jiankaiwang/devops/Fruit_Recognition/eval/qnap_fruit_val_00003.JPEG'
image_path = '/notebooks/devops/Auxiliary_Operations/data/IMG_1562_single_crop/screw/IMG_1562_frame_68_597213347968410b3e1b212b7c93a320767da9a1.jpg'
output_dict = single_image(image_path)
print(output_dict)

{'final_result': array([[3.6213717e-06, 9.9999642e-01]], dtype=float32)}


In [10]:
cls_idx = int(np.argmax(output_dict['final_result'], axis=1) + 1)
print("Classification: {}".format(category_index[cls_idx]))

Classification: screw
